In [1]:
import os

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'D:/Suraj/Cloud/Google Cloud/llm-learning-with-jp-7cd0cee5b01f.json'

***

In [4]:
# !pip install pandas sqlalchemy -qU

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("./penguins.csv")

In [7]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [8]:
from sqlalchemy import create_engine, text

In [9]:
temp_db = create_engine(url = "sqlite:///:memory:", echo = True)

In [10]:
data = df.to_sql(name = "Penguins", con=temp_db)

2024-01-14 09:04:24,724 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-14 09:04:24,730 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Penguins")
2024-01-14 09:04:24,732 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-14 09:04:24,732 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Penguins")
2024-01-14 09:04:24,732 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-14 09:04:24,739 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Penguins" (
	"index" BIGINT, 
	species TEXT, 
	island TEXT, 
	bill_length_mm FLOAT, 
	bill_depth_mm FLOAT, 
	flipper_length_mm FLOAT, 
	body_mass_g FLOAT, 
	sex TEXT
)


2024-01-14 09:04:24,739 INFO sqlalchemy.engine.Engine [no key 0.00107s] ()
2024-01-14 09:04:24,744 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Penguins_index" ON "Penguins" ("index")
2024-01-14 09:04:24,744 INFO sqlalchemy.engine.Engine [no key 0.00113s] ()
2024-01-14 09:04:24,755 INFO sqlalchemy.engine.Engine INSERT INTO "Penguins" ("index", species, island, bill_length_m

## Connect to SQL DataBase

Using SQL Alchemy we can establish a connection to this temporary database and query it for the results:

In [11]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT * from Penguins WHERE island='Torgersen'"))

2024-01-14 09:07:13,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-14 09:07:13,453 INFO sqlalchemy.engine.Engine SELECT * from Penguins WHERE island='Torgersen'
2024-01-14 09:07:13,457 INFO sqlalchemy.engine.Engine [generated in 0.00265s] ()
2024-01-14 09:07:13,459 INFO sqlalchemy.engine.Engine ROLLBACK


In [13]:
result.all()

[(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
 (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE'),
 (2, 'Adelie', 'Torgersen', 40.3, 18.0, 195.0, 3250.0, 'FEMALE'),
 (3, 'Adelie', 'Torgersen', None, None, None, None, None),
 (4, 'Adelie', 'Torgersen', 36.7, 19.3, 193.0, 3450.0, 'FEMALE'),
 (5, 'Adelie', 'Torgersen', 39.3, 20.6, 190.0, 3650.0, 'MALE'),
 (6, 'Adelie', 'Torgersen', 38.9, 17.8, 181.0, 3625.0, 'FEMALE'),
 (7, 'Adelie', 'Torgersen', 39.2, 19.6, 195.0, 4675.0, 'MALE'),
 (8, 'Adelie', 'Torgersen', 34.1, 18.1, 193.0, 3475.0, None),
 (9, 'Adelie', 'Torgersen', 42.0, 20.2, 190.0, 4250.0, None),
 (10, 'Adelie', 'Torgersen', 37.8, 17.1, 186.0, 3300.0, None),
 (11, 'Adelie', 'Torgersen', 37.8, 17.3, 180.0, 3700.0, None),
 (12, 'Adelie', 'Torgersen', 41.1, 17.6, 182.0, 3200.0, 'FEMALE'),
 (13, 'Adelie', 'Torgersen', 38.6, 21.2, 191.0, 3800.0, 'MALE'),
 (14, 'Adelie', 'Torgersen', 34.6, 21.1, 198.0, 4400.0, 'MALE'),
 (15, 'Adelie', 'Torgersen', 36.6, 17.

## Using LLMs to Generate SQL from Natural Language

First, let's create a prefix we can add to our prompts that helps the model understand the table:

In [14]:
def create_prefix(query): 
    
    prefix=f"""
    Return a SQL statement that answers the following query:
    {query}

    For a table called 'Penguins' with the following properties:
    
     #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object 
     
    Example Rows:
    (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
    (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')
    
    Only return the SQL statement for the query. 
    """ 
    return prefix

In [35]:
def user_input(return_query_only=False):
    query = input("Ask a question about the database :-->\n")
    if return_query_only:
        return query
    else:
        return create_prefix(query)

In [37]:
prefix = user_input()

Ask a question about the database :-->
 How many Penguins are female?


In [38]:
print(prefix)


    Return a SQL statement that answers the following query:
    How many Penguins are female?

    For a table called 'Penguins' with the following properties:
    
     #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object 
     
    Example Rows:
    (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
    (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')
    
    Only return the SQL statement for the query. 
    


***

In [20]:
from vertexai.language_models import CodeGenerationModel

In [21]:
codegen_model = CodeGenerationModel.from_pretrained('code-bison')

In [22]:
print("Hello! I am your AI Assistant. Ask me anything about the database")
print("\n\n")
prefix = user_input()
result = codegen_model.predict(prefix=prefix)

Hello! I am your AI Assistant. Ask me anything about the database





Ask a question about the database :-->
 How many Penguins are female?


In [23]:
result

```sql
SELECT COUNT(*)
FROM Penguins
WHERE sex = 'FEMALE';
```

In [24]:
result.text

"```sql\nSELECT COUNT(*)\nFROM Penguins\nWHERE sex = 'FEMALE';\n```"

***

### Clean Text

In [27]:
def clean_sql(sql_statement):
    return sql_statement.replace('```sql','').replace("```",'').replace('\n',' ').strip()

In [28]:
clean_sql(result.text)

"SELECT COUNT(*) FROM Penguins WHERE sex = 'FEMALE';"

### Combine everything together

In [29]:
def nlp_assistant():
    print("Hello! I am your AI Assistant. Ask me anything about the database")
    print("\n\n")
    prefix = user_input()
    result = codegen_model.predict(prefix=prefix)
    sql = clean_sql(result.text.strip())
    with temp_db.connect() as conn:
        print("\nOk, I am running this SQL Statement")
        result = conn.execute(text(sql))
    print('-------')
    print('-------')
    print("Here is the answer to your query")
    print('\n')
    print(result.all())

In [31]:
nlp_assistant()

Hello! I am your AI Assistant. Ask me anything about the database





Ask a question about the database :-->
 How many penguins are female ?



Ok, I am running this SQL Statement
2024-01-14 09:24:14,827 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-14 09:24:14,827 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM Penguins WHERE sex = 'FEMALE';
2024-01-14 09:24:14,832 INFO sqlalchemy.engine.Engine [generated in 0.00314s] ()
2024-01-14 09:24:14,834 INFO sqlalchemy.engine.Engine ROLLBACK
-------
-------
Here is the answer to your query


[(165,)]


In [32]:
nlp_assistant()

Hello! I am your AI Assistant. Ask me anything about the database





Ask a question about the database :-->
 how many uniquely island are here



Ok, I am running this SQL Statement
2024-01-14 09:25:00,707 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-14 09:25:00,707 INFO sqlalchemy.engine.Engine SELECT COUNT(DISTINCT island) FROM Penguins;
2024-01-14 09:25:00,707 INFO sqlalchemy.engine.Engine [generated in 0.00267s] ()
2024-01-14 09:25:00,713 INFO sqlalchemy.engine.Engine ROLLBACK
-------
-------
Here is the answer to your query


[(3,)]


In [33]:
nlp_assistant()

Hello! I am your AI Assistant. Ask me anything about the database





Ask a question about the database :-->
 What are the unique island names ?



Ok, I am running this SQL Statement
2024-01-14 09:25:55,592 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-14 09:25:55,595 INFO sqlalchemy.engine.Engine SELECT DISTINCT island FROM Penguins;
2024-01-14 09:25:55,596 INFO sqlalchemy.engine.Engine [generated in 0.00331s] ()
2024-01-14 09:25:55,598 INFO sqlalchemy.engine.Engine ROLLBACK
-------
-------
Here is the answer to your query


[('Torgersen',), ('Biscoe',), ('Dream',)]


***
***

In [41]:
from vertexai.language_models import TextGenerationModel

In [42]:
model = TextGenerationModel.from_pretrained('text-bison')

In [55]:
def nlp_assistant():
    print("Hello! I am your AI Assistant. Ask me anything about the database")
    print("\n\n")
    user_query = user_input(return_query_only=True)
    prefix = create_prefix(user_query)
    result = codegen_model.predict(prefix=prefix)
    sql = clean_sql(result.text.strip())
    with temp_db.connect() as conn:
        print("\nOk, I am running this SQL Statement")
        result = conn.execute(text(sql))
    print('-------')
    print('-------')
    print("Here is the answer to your query")
    print('\n')

    prompt = f"""Given a question with some context you have to answer the question.
    You have extract the information from context very carefully to answer the question

    Question:  How many penguins are there in the dataset?
    Context : [(100,)]
    Answer: There is 100 penguin in the dataset.
    
    Question: {user_query}
    Context : {result.all()}
    """
    # print(prompt)
    # print(result.all())
    response = model.predict(prompt=prompt)
    print(response.text)

In [57]:
nlp_assistant()

Hello! I am your AI Assistant. Ask me anything about the database





Ask a question about the database :-->
 Provide the count of male and female penguins.



Ok, I am running this SQL Statement
2024-01-14 09:39:43,722 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-14 09:39:43,722 INFO sqlalchemy.engine.Engine SELECT sex, COUNT(*) AS count FROM Penguins WHERE sex IS NOT NULL GROUP BY sex;
2024-01-14 09:39:43,722 INFO sqlalchemy.engine.Engine [generated in 0.00281s] ()
2024-01-14 09:39:43,722 INFO sqlalchemy.engine.Engine ROLLBACK
-------
-------
Here is the answer to your query


 Answer: There are 165 female and 168 male penguins.


In [58]:
nlp_assistant()

Hello! I am your AI Assistant. Ask me anything about the database





Ask a question about the database :-->
 Which penguin has the max body_mass_g ?



Ok, I am running this SQL Statement
2024-01-14 09:40:47,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-14 09:40:47,453 INFO sqlalchemy.engine.Engine SELECT species, MAX(body_mass_g) AS max_body_mass_g FROM Penguins GROUP BY species ORDER BY max_body_mass_g DESC LIMIT 1;
2024-01-14 09:40:47,453 INFO sqlalchemy.engine.Engine [generated in 0.00306s] ()
2024-01-14 09:40:47,453 INFO sqlalchemy.engine.Engine ROLLBACK
-------
-------
Here is the answer to your query


 Answer: Gentoo penguin has the max body_mass_g of 6300.0.
